In [38]:
"""
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
"""

'\n%tensorflow_version 2.x\nimport tensorflow as tf\nprint("Tensorflow version " + tf.__version__)\n\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  tf.config.experimental_connect_to_cluster(tpu)\n  tf.tpu.experimental.initialize_tpu_system(tpu)\n  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)\n  print(\'Running on TPU \', tpu.cluster_spec().as_dict()[\'worker\'])\nexcept ValueError:\n  raise BaseException(\'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!\')\n'

In [39]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
name   ="M1_2008-2022_DataFrame"
#df=pd.read_csv("EURUSD.csv")
#df = pd.read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python',index_col=[0])
    
print("Number of rows and columns:", df.shape)
#df = df.drop('Adj Close',axis=1)
#df = df.drop('Volume',axis=1)
df.tail(10)

Number of rows and columns: (4871857, 5)


,Date,Open,High,Low,Close
4871847,2022-02-25 23:45:00,1.12724,1.12729,1.12721,1.12724
4871848,2022-02-25 23:46:00,1.12723,1.12723,1.12715,1.12717
4871849,2022-02-25 23:47:00,1.12715,1.12719,1.12712,1.12717
4871850,2022-02-25 23:48:00,1.12718,1.12734,1.12712,1.12729
4871851,2022-02-25 23:49:00,1.12728,1.12730,1.12717,1.12727
4871852,2022-02-25 23:50:00,1.12726,1.12728,1.12714,1.12718
4871853,2022-02-25 23:51:00,1.12716,1.12720,1.12712,1.12712
4871854,2022-02-25 23:52:00,1.12716,1.12718,1.12713,1.12715
4871855,2022-02-25 23:53:00,1.12715,1.12728,1.12711,1.12725
4871856,2022-02-25 23:54:00,1.12726,1.12726,1.12714,1.12716


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:train_size, 1:].values, df.iloc[:train_size, 1:].values, test_size=0.01)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4340824, 4), (43847, 4), (4340824, 4), (43847, 4))

In [41]:
"""
train_size = ((len(df)*90)//100)
training_set = df.iloc[:train_size, 1:2].values
test_set = df.iloc[train_size:, 1:2].values

training_set[:5],test_set[:5]
"""

'\ntrain_size = ((len(df)*90)//100)\ntraining_set = df.iloc[:train_size, 1:2].values\ntest_set = df.iloc[train_size:, 1:2].values\n\ntraining_set[:5],test_set[:5]\n'

In [42]:
"""
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

time_steps_train =1 #60

# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(time_steps_train, train_size):
    X_train.append(training_set_scaled[i-time_steps_train:i, 0])
    y_train.append(training_set_scaled[i, 0])
    print(X_train,y_train,i-time_steps_train,i)
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)
print(y_train.shape)
print(X_train[-5:])
print(y_train[-5:])
#(740, 60, 1)
"""

'\n# Feature Scaling\nsc = MinMaxScaler(feature_range = (0, 1))\ntraining_set_scaled = sc.fit_transform(training_set)\n\ntime_steps_train =1 #60\n\n# Creating a data structure with 60 time-steps and 1 output\nX_train = []\ny_train = []\nfor i in range(time_steps_train, train_size):\n    X_train.append(training_set_scaled[i-time_steps_train:i, 0])\n    y_train.append(training_set_scaled[i, 0])\n    print(X_train,y_train,i-time_steps_train,i)\nX_train, y_train = np.array(X_train), np.array(y_train)\nX_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))\nprint(X_train.shape)\nprint(y_train.shape)\nprint(X_train[-5:])\nprint(y_train[-5:])\n#(740, 60, 1)\n'

In [43]:
"""# Getting the predicted stock price of 2017
dataset_train = df.iloc[:train_size, 1:2]
dataset_test = df.iloc[train_size:, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_steps_train:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
y_test = []
for i in range(time_steps_train, len(df)-train_size+time_steps_train): #519
    X_test.append(inputs[i-time_steps_train:i, 0])
    y_test.append(inputs[i, 0])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test.shape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))

print(X_test.shape)
print(y_test.shape)
print(X_test[-5:])
print(y_test[-5:])

#(459, 60, 1)
"""

'# Getting the predicted stock price of 2017\ndataset_train = df.iloc[:train_size, 1:2]\ndataset_test = df.iloc[train_size:, 1:2]\ndataset_total = pd.concat((dataset_train, dataset_test), axis = 0)\ninputs = dataset_total[len(dataset_total) - len(dataset_test) - time_steps_train:].values\ninputs = inputs.reshape(-1,1)\ninputs = sc.transform(inputs)\nX_test = []\ny_test = []\nfor i in range(time_steps_train, len(df)-train_size+time_steps_train): #519\n    X_test.append(inputs[i-time_steps_train:i, 0])\n    y_test.append(inputs[i, 0])\nX_test = np.array(X_test)\ny_test = np.array(y_test)\nX_test.shape\nX_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))\n#y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))\n\nprint(X_test.shape)\nprint(y_test.shape)\nprint(X_test[-5:])\nprint(y_test[-5:])\n\n#(459, 60, 1)\n'

In [44]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 4))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

# Fitting the RNN to the Training set

In [ ]:
model.fit(X_train, y_train, epochs = 5, batch_size = 4096,validation_data=(X_test,y_test))

Epoch 1/5
1060/1060 [==============================] - 273s 251ms/step - loss: 0.0157 - accuracy: 0.2483 - val_loss: 3.5747e-05 - val_accuracy: 0.1127
Epoch 2/5
  55/1060 [>.............................] - ETA: 4:11 - loss: 0.0029 - accuracy: 0.2540

In [ ]:
predicted_stock_price = model.predict(X_test)
#predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price[-5:],y_test[-5:]

In [ ]:
# Visualising the results
plt.plot(df.loc[train_size:, "Date"],y_test.values, color = "red", label = "Real Price")
plt.plot(df.loc[train_size:, "Date"],predicted_stock_price, color = "blue", label = "Predicted Price")
#plt.xticks(np.arange(0,len(df),50))
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
dataset_test-predicted_stock_price